<a href="https://colab.research.google.com/github/srikarraju/eGrocery_Demand_Prediction/blob/main/NAC_eGrocery_1_product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorboardX

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import MultivariateNormal,Categorical
import random
import pandas as pd
from tensorboardX import SummaryWriter

In [ ]:
class EGroceryEnv():

	def __init__(self, df=pd.DataFrame({0:[0]}), products_count=10, features=['a'], shelf_life=[1], wastage_cost=[1], shortage_cost=[1]):
		super(EGroceryEnv, self).__init__()

		self.df = df
		self.products_count = products_count
		self.shelf_life = shelf_life
		self.features = features
		self.wastage_cost = wastage_cost
		self.shortage_cost = shortage_cost
		self.current_step = 0
		self.wastage_track = list([])
		self.shortage_track = list([])
		self.reward_track = list([])

		#variables to track shartage and wastage
		self.shortage = np.array(list([0]*self.products_count))
		self.wastage = np.array(list([0]*self.products_count))

		#Define Stock
		self.stock = list([])
		for i in range(self.products_count):
			self.stock.append([0])
		print(features)


	def _next_observation(self):

		obs  = self.df.loc[self.current_step,self.features]

		st_temp = list([])
		for i in range(len(self.stock)):
			for j in range(1,min(int(self.stock[i][0])+1,5)):
				st_temp.append(self.stock[i][j])
			if(self.stock[i][0]==5):
				st_temp.append(self.stock[i][4])
			elif(self.stock[i][0]<5):
				for j in range(int(self.stock[i][0])+1,6):
					st_temp.append(0)
			else:
				st_temp.append(np.sum(self.stock[i][5:int(self.stock[i][0])+1]))
		obs = list(obs) + list(st_temp) + list(self.shelf_life)
		return obs

	def _take_action(self, action):
		#Add products to the current stocks
		for i in range(self.products_count):
			if(len(self.stock[i])<self.shelf_life[i]):
				for j in range(len(self.stock[i]),int(self.shelf_life[i])):
					self.stock[i].append(0)
			self.stock[i].append(action[i])
			self.stock[i][0]=self.shelf_life[i]



		#Fullfill demand
		prods = ['prod'+str(i) for i in [8,11,15,17,94,95,96,110,112,128]]
		demand = self.df.loc[self.current_step+1,prods]
		for i in range(self.products_count):
			for j in range(1,int(self.stock[i][0])+1):
				if(self.stock[i][j]>=demand[i]):
					self.stock[i][j] = self.stock[i][j] - demand[i]
					demand[i] = 0
					break
				else:
					demand[i] = demand[i] - self.stock[i][j]
					self.stock[i][j] = 0
			if(demand[i]>0):
				self.shortage[i]=demand[i]

		#Update shelf life and find out wastage
		for i in range(self.products_count):
			self.stock[i][0] = self.stock[i][0] -1
			if(self.stock[i][1]>0):
				self.wastage[i] = self.stock[i][1]
			for j in range(1,int(self.stock[i][0])+1):
				self.stock[i][j] = self.stock[i][j+1]
			self.stock[i].pop()


	def step(self, action):
	        # update stock, fullfill demand and calculate shortage and wastage
		quantity = [6, 10, 15, 4, 6, 2, 7, 50, 2, 30]
		action1 = [0]*self.products_count
		for i in range(len(action)):
			action1[i] = action[i]*quantity[i]
		self._take_action(action1)
		self.action = action

		#increment step
		self.current_step += 1


		reward = -1*(np.matmul(self.wastage_cost,self.wastage.transpose())+np.matmul(self.shortage_cost,self.shortage.transpose()))
		self.reward = reward
		done = (self.current_step < 0) or (self.current_step > self.df.shape[0]-2)

		obs = self._next_observation()

		self.wastage_track.append(np.sum(self.wastage))
		self.shortage_track.append(np.sum(self.shortage))
		self.reward_track.append(np.abs(self.reward))


		self.shortage = np.array(list([0]*self.products_count))
		self.wastage = np.array(list([0]*self.products_count))

		return obs, np.sum(self.reward), done, {}

	def reset(self):
		# Reset the state of the environment to an initial state
		self.current_step = 0
		self.shortage = np.array(list([0]*self.products_count))
		self.wastage = np.array(list([0]*self.products_count))
		self.stock = list([])
		for i in range(self.products_count):
			self.stock.append([0])

		#print(len(self.features),products_count)
		#print(self.features)
		return [0]*len(self.features) + [0]*6*self.products_count

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RL_Project/Models/ppo_based/PPOBased/data/final_data_trainx.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RL_Project/Models/ppo_based/PPOBased/data/final_data_testx.csv')

# products_count = 10
products_count = 1

# avg_f7 = ['prod'+str(i)+'avg7' for i in [8,11,15,17,94,95,96,110,112,128]]
# avg_f15 = ['prod'+str(i)+'avg15' for i in [8,11,15,17,94,95,96,110,112,128]]
# avg_f30 = ['prod'+str(i)+'avg30' for i in [8,11,15,17,94,95,96,110,112,128]]

avg_f7 = ['prod'+str(i)+'avg7' for i in [8]]
avg_f15 = ['prod'+str(i)+'avg15' for i in [8]]
avg_f30 = ['prod'+str(i)+'avg30' for i in [8]]

features = ['month', 'monthday', 'weekday'] + avg_f7 + avg_f15 + avg_f30

print(features)

# shelf_life = np.array([4, 3, 5, 10, 7, 2, 1, 3, 8, 6], dtype=np.float32)
shelf_life = np.array([4], dtype=np.float32)

wastage_cost = np.array([1]*products_count, dtype=np.float16)
shortage_cost = np.array([1]*products_count, dtype=np.float16)

action_std = 0.1
eps_clip = 0.2
gamma = 0.99

lr = 0.0001
betas = (0.9, 0.999)
K_epochs = 5

update_timestep = 20
time_step=0
running_reward = 0

state_dim = len(features) + 6*products_count
action_dim = products_count

env = EGroceryEnv(df_train, products_count, features, shelf_life, wastage_cost, shortage_cost)

writer = SummaryWriter()

Total_reward = []


['month', 'monthday', 'weekday', 'prod8avg7', 'prod8avg15', 'prod8avg30']
['month', 'monthday', 'weekday', 'prod8avg7', 'prod8avg15', 'prod8avg30']


In [ ]:
class policy_net(nn.Module):
  def __init__(self,state_dim,hidden_dim,action_dim):
    super(policy_net,self).__init__()
    self.h = nn.Linear(state_dim,hidden_dim)
    self.out = nn.Linear(hidden_dim,action_dim)
    self.action_var = torch.Tensor([0.2]*1)

  def forward(self,x):
    x = F.relu(self.h(x))
    x = F.softmax(self.out(x),dim=1)
    return x

  def act(self,action_probs):
    dist = Categorical(action_probs)
    action = dist.sample()
    action_logprob = dist.log_prob(action)

    return action.detach(), action_logprob

In [ ]:
weights_v = np.zeros(12,dtype=float)
policy = policy_net(12,100,10)
optimizer = torch.optim.SGD(policy.parameters(),lr=1)

In [ ]:
# ALGORITHM 1
alpha_0, beta_0, gamma, epsilon = 0.1, 0.01, 0.95, 0.1
alpha_c, beta_c = 1000, 100000
t = 0
n_episode = 0
actions_list = []
avg_reward = 0

while n_episode <=10000:
  rewards,states,actions = [],[],[]
  state = env.reset()
  total_reward = 0
  while True:
    action_probs = policy(torch.tensor(state).unsqueeze(0).float()).squeeze(0)
    print(action_probs)
    action,action_logprob = policy.act(action_probs)
    print(action)
    new_state, reward, done,info = env.step([action.item()])

    value_curr = np.dot(weights_v,state)
    value_next = np.dot(weights_v,new_state)

    beta = (beta_0 * beta_c) / (beta_c + t)
    alpha = (alpha_0 * alpha_c) / (alpha_c + t**(2/3))

    avg_reward = (1 - alpha*gamma)* avg_reward + gamma * alpha* reward
    td_error = reward + value_curr - value_next - avg_reward

    weights_v += 0.01*alpha*abs(td_error)/td_error * np.asarray(state)

    policy_net_loss = -0.01*beta*abs(td_error)/td_error*action_logprob
    optimizer.zero_grad()
    policy_net_loss.backward()
    optimizer.step()

    total_reward += reward

    state = new_state
    if done == 1:
      break
  print("Epoch:",n_episode,"Reward:",total_reward)

  n_episode += 1

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       grad_fn=<SqueezeBackward1>)


ValueError: ignored

In [ ]:
env2 = EGroceryEnv(df_test, products_count, features, shelf_life, wastage_cost, shortage_cost)
state = env2.reset()
actions_test = []
rewards_test = []
for step in range(0,30):
  #print("state:",state)
  action_mean = policy(torch.tensor(state).unsqueeze(0).float()).squeeze(0)
  action,action_logprob = policy.act(state,action_mean)
  new_state, reward, done,info = env2.step(action)
  actions_test.append(action.item())
  rewards_test.append(reward)
  state = new_state
df_test_output = pd.DataFrame()
df_test_output['rewards'] = rewards_test
df_test_output['actions'] = actions_test
df_test_output.to_csv("df_test_output")

print(sum(rewards_test))
print(actions_test)

['month', 'monthday', 'weekday', 'prod8avg7', 'prod8avg15', 'prod8avg30']
-411.0
[1.1278609037399292, 0.9951709508895874, 1.8461501598358154, 1.265052318572998, 1.3535690307617188, 1.498424768447876, 0.03568905591964722, 0.7335294485092163, 0.9459399580955505, 1.0868159532546997, 0.7744569778442383, 1.5525164604187012, 0.510865330696106, 0.40411025285720825, 0.9203477501869202, 0.9871135950088501, 0.9015095829963684, 0.1721564531326294, 0.5283150672912598, 1.1966784000396729, 1.9170793294906616, 0.6862308979034424, 0.8935506343841553, 1.4949328899383545, 0.7906617522239685, 0.7753675580024719, 0.6639062166213989, 1.5138680934906006, 1.5781338214874268, 1.3377673625946045]


In [ ]:
env2 = EGroceryEnv(df_test, products_count, features, shelf_life, wastage_cost, shortage_cost)
state = env2.reset()
actions_test = [0.7320775389671326, 0.7109668254852295, 0.3982686996459961, 0.42593663930892944, 0.9960396885871887, 0.5905162692070007, 1.6885946989059448, 0.36574244499206543, 0.9635764360427856, 1.1523358821868896, 0.7376989126205444, 1.246662974357605, 1.4320002794265747, 1.0723148584365845, 0.554922878742218, 1.592714786529541, 1.4837706089019775, 1.2334750890731812, 1.6866092681884766, 1.206745982170105, 1.2877410650253296, 1.1382571458816528, 1.1183135509490967, 0.5739326477050781, 0.36914557218551636, 1.4445385932922363, 0.5771536827087402, 0.5141178369522095, 1.4803128242492676, 0.2578907608985901]
rewards_test = []
for step in range(0,30):
  #print("state:",state)
  new_state, reward, done,info = env2.step([actions_test[step]])
  rewards_test.append(reward)
  state = new_state
df_test_output = pd.DataFrame()
df_test_output['rewards'] = rewards_test
df_test_output['actions'] = actions_test
df_test_output.to_csv("df_test_output")

print(sum(rewards_test))
print(actions_test)
print(rewards_test)

['month', 'monthday', 'weekday', 'prod8avg7', 'prod8avg15', 'prod8avg30']
-418.0
[0.7320775389671326, 0.7109668254852295, 0.3982686996459961, 0.42593663930892944, 0.9960396885871887, 0.5905162692070007, 1.6885946989059448, 0.36574244499206543, 0.9635764360427856, 1.1523358821868896, 0.7376989126205444, 1.246662974357605, 1.4320002794265747, 1.0723148584365845, 0.554922878742218, 1.592714786529541, 1.4837706089019775, 1.2334750890731812, 1.6866092681884766, 1.206745982170105, 1.2877410650253296, 1.1382571458816528, 1.1183135509490967, 0.5739326477050781, 0.36914557218551636, 1.4445385932922363, 0.5771536827087402, 0.5141178369522095, 1.4803128242492676, 0.2578907608985901]
[-5.0, -2.0, -3.0, -3.0, -3.0, -49.0, -40.0, -12.0, -45.0, -2.0, -1.0, -4.0, 0.0, 0.0, 0.0, -37.0, -30.0, -29.0, -17.0, -23.0, -21.0, -3.0, -6.0, -9.0, -8.0, -10.0, -4.0, -16.0, -11.0, -25.0]


In [ ]:
AC_LSTM = rewards_test
for i in range(len(AC_LSTM)):
  AC_LSTM[i] = AC_LSTM[i]
print(AC_LSTM)

[2.0, -0.0, -0.0, -0.0, -0.0, 39.0, 50.0, 10.0, 45.0, 2.0, 1.0, 2.0, 3.0, -0.0, -0.0, 44.0, 34.0, 35.0, 24.0, 23.0, 17.0, 5.0, 8.0, 4.0, 6.0, 14.0, 3.0, 10.0, 11.0, 19.0]
